# Práctica 2 

Elaboró: Carlos Alejandro Jarero Gonzalez <al255813@alumnos.uacj.mx>

Matrícula: 255813

El presente Notebook fue relizado en equipo local con Kernel Python 3.11.8 en VS Code.

## Instrucciones

Este notebook puede extraer datos de correos que se envían con los precios de las acciones del archivo ```informe_acciones.txt```, para obtener los tickers, fechas del precio, nombre de la compañia y precio de la acción. Con estos datos posteriormente por acción se calcula la media, varaianza, desviación estándar, mínimo y máximo; para mostrarlo en pantalla y archivos de salida.

Si quieres cambiar el el archivo solo ve a la sección de parameters y cambia el ```input_path```. Ten en cuenta que este notebook genera un archivo con resultados por lo tanto el parametro ``output_path`` debde de llenarse correctamente, la carpeta donde se almacena el resultado debe de exisitr. El nombre del archivo de salida se generará automáticamente, se generan dos ```result-*.txt``` con la tabla sin estadísticas completas y ```stats-*.txt``` con la misma tabla con un complemento estadístico.

In [177]:
from pathlib import Path
from datetime import datetime
from tabulate import tabulate
import numpy as np 
import re

## Sección de parámetros

In [178]:
# Parameters
input_path = Path("./ClassFiles/informe_acciones.txt")
output_path = "./output/"


## Extracción

In [179]:
raw_emails = []

with open(input_path, "r", encoding="cp1252") as file:
    raw_email = ""
    for line in file:
        if re.search(r"^\n$",line) != None:
            continue
        if re.search(r"^-{40}$",line) == None:
            raw_email += line
        else:
            raw_emails.append(raw_email)
            raw_email = ""

extracted_data = {}

for email in raw_emails:
    ticker = re.findall(r"\(([A-Z0-9.]{1,7})\)", email)[0]
    date = re.findall(r"(\d{4}-\d{2}-\d{2})", email)[0] 
    price = float(re.findall(r"\$(\d+(?:\.\d+)?)", email)[0])
    company = re.findall(r"acci.n de*\s(.+)\s*\(", email)[0].strip()

    if extracted_data.get(ticker) == None:
        extracted_data[ticker] = []
        extracted_data[ticker] = {
            "company": company,
            "ticker": ticker,
            "last_date": None,
            "average": None,
            "variance": None,
            "sd": None,
            "min": None,
            "max": None,
            "prices": [],
            "dates": []
        }

    extracted_data[ticker]["prices"].append(price)
    extracted_data[ticker]["dates"].append(datetime.strptime(date, r"%Y-%m-%d").date())

for ticker_code in extracted_data.keys():
    extracted_data[ticker_code]["average"] = np.average(extracted_data[ticker_code]["prices"])
    extracted_data[ticker_code]["variance"] = np.var(extracted_data[ticker_code]["prices"])
    extracted_data[ticker_code]["sd"] = np.std(extracted_data[ticker_code]["prices"])
    extracted_data[ticker_code]["min"] = np.min(extracted_data[ticker_code]["prices"])
    extracted_data[ticker_code]["max"] = np.max(extracted_data[ticker_code]["prices"])
    extracted_data[ticker_code]["last_date"] = np.max(extracted_data[ticker_code]["dates"])

result_table_data = []
result_table_headers = ["Empresa", "Fecha más reciente", "Valor Máx ($)", "Valor Mín ($)", "Promedio ($)"]

for ticker_code in extracted_data.keys():
    result_table_data.append([
        extracted_data[ticker_code]["company"], 
        extracted_data[ticker_code]["last_date"],
        round(extracted_data[ticker_code]["max"],2),
        round(extracted_data[ticker_code]["min"],2),
        round(extracted_data[ticker_code]["average"],2)
    ])


formated_result_table = tabulate(result_table_data, result_table_headers, tablefmt="grid", floatfmt=".2f")

result_table = Path(output_path + f"result-{input_path.stem}-{datetime.now().strftime(r'%Y%m%d%H%M%S')}.txt")

with open(result_table, "w") as file:
    file.write(formated_result_table)
    print(f"The stats are saved in: {result_table}")


print(formated_result_table)



The stats are saved in: output/result-informe_acciones-20250126121938.txt
+---------------------------------+----------------------+-----------------+-----------------+----------------+
| Empresa                         | Fecha más reciente   |   Valor Máx ($) |   Valor Mín ($) |   Promedio ($) |
+=================================+======================+=================+=================+================+
| Cisco Systems Inc.              | 2024-09-02           |         1499.73 |          104.72 |         794.88 |
+---------------------------------+----------------------+-----------------+-----------------+----------------+
| Amazon.com Inc.                 | 2024-09-02           |         1498.27 |          103.83 |         762.75 |
+---------------------------------+----------------------+-----------------+-----------------+----------------+
| NVIDIA Corporation              | 2024-09-02           |         1496.06 |          103.26 |         806.61 |
+-----------------------------

In [180]:

stats_table_data = []
stats_table_headers = ["Empresa", "Fecha más reciente", "Valor Máx ($)", "Valor Mín ($)", "Promedio ($)", "Varianza ($^2$)", "Std.Dev ($)"]

for ticker_code in extracted_data.keys():
    stats_table_data.append([
        extracted_data[ticker_code]["company"], 
        extracted_data[ticker_code]["last_date"],
        round(extracted_data[ticker_code]["max"],2),
        round(extracted_data[ticker_code]["min"],2),
        round(extracted_data[ticker_code]["average"],2),
        round(extracted_data[ticker_code]["variance"],2),
        round(extracted_data[ticker_code]["sd"],2)
    ])

formated_stats_table = tabulate(stats_table_data, stats_table_headers, tablefmt="grid", floatfmt=".2f")

stats_table = Path(output_path + f"stats-{input_path.stem}-{datetime.now().strftime(r'%Y%m%d%H%M%S')}.txt")

with open(stats_table, "w") as file:
    file.write(formated_stats_table)
    print(f"The stats are saved in: {stats_table}")

print(formated_stats_table)

The stats are saved in: output/stats-informe_acciones-20250126121938.txt
+---------------------------------+----------------------+-----------------+-----------------+----------------+-------------------+---------------+
| Empresa                         | Fecha más reciente   |   Valor Máx ($) |   Valor Mín ($) |   Promedio ($) |   Varianza ($^2$) |   Std.Dev ($) |
+=================================+======================+=================+=================+================+===================+===============+
| Cisco Systems Inc.              | 2024-09-02           |         1499.73 |          104.72 |         794.88 |         168425.29 |        410.40 |
+---------------------------------+----------------------+-----------------+-----------------+----------------+-------------------+---------------+
| Amazon.com Inc.                 | 2024-09-02           |         1498.27 |          103.83 |         762.75 |         162284.02 |        402.84 |
+---------------------------------+----